In [105]:
import pandas as pd
from datetime import date
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM,Activation,RepeatVector
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

In [106]:
df = pd.read_csv('m-hsmort7112.txt', sep=' ')

In [107]:
df.head()

,year,mon,hs,mort
0,1971,4,1986,7.31
1,1971,5,2049,7.43
2,1971,6,2026,7.53
3,1971,7,2083,7.60
4,1971,8,2158,7.70


In [108]:
df['Date'] = df['year'].astype(str)+' /'+df['mon'].astype(str)

In [109]:
df_new = df[['Date', 'hs','mort']]
df_new

,Date,hs,mort
0,1971 /4,1986,7.31
1,1971 /5,2049,7.43
2,1971 /6,2026,7.53
3,1971 /7,2083,7.60
4,1971 /8,2158,7.70
...,...,...,...
487,2011 /11,702,3.99
488,2011 /12,681,3.96
489,2012 /1,714,3.92
490,2012 /2,694,3.89


In [111]:
data_set = df_new[['hs','mort']]
data_set

,hs,mort
0,1986,7.31
1,2049,7.43
2,2026,7.53
3,2083,7.60
4,2158,7.70
...,...,...
487,702,3.99
488,681,3.96
489,714,3.92
490,694,3.89


In [137]:
#Normalizing the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_dataset = scaler.fit_transform(data_set)
scaled_dataset

array([[0.74801587, 0.23489011],
       [0.77926587, 0.24313187],
       [0.76785714, 0.25      ],
       [0.79613095, 0.25480769],
       [0.83333333, 0.26167582],
       [0.77529762, 0.26098901],
       [0.81845238, 0.25686813],
       [0.8452381 , 0.25137363],
       [0.90128968, 0.24656593],
       [1.        , 0.24381868],
       [0.9484127 , 0.23626374],
       [0.92063492, 0.2342033 ],
       [0.87847222, 0.23351648],
       [0.86458333, 0.23901099],
       [0.88095238, 0.23901099],
       [0.87996032, 0.24107143],
       [0.94444444, 0.24107143],
       [0.99355159, 0.24244505],
       [0.99553571, 0.24244505],
       [0.96378968, 0.24313187],
       [0.93650794, 0.24381868],
       [0.99355159, 0.24381868],
       [0.89831349, 0.24381868],
       [0.9360119 , 0.24519231],
       [0.79662698, 0.25068681],
       [0.88690476, 0.25824176],
       [0.78819444, 0.26373626],
       [0.81597222, 0.28571429],
       [0.78025794, 0.31662088],
       [0.69246032, 0.3385989 ],
       [0.

In [132]:
def create_dataset(scaled_data):
    window_size = 4
    x_set =[]
    y_set =[]
    
    for i in range(window_size,len(scaled_data)):
        x_set.append(scaled_data[i-window_size:i,:])
        y_set.append(scaled_data[i,0:1])
        
    return x_set, y_set

In [138]:
#get the x data and y data 
x_set, y_set = create_dataset(scaled_dataset)
x_data = np.array(x_set)
y_data = np.array(y_set)
print(x_data.shape)
print(y_data.shape)
y_data

(488, 4, 2)
(488, 1)


array([[0.83333333],
       [0.77529762],
       [0.81845238],
       [0.8452381 ],
       [0.90128968],
       [1.        ],
       [0.9484127 ],
       [0.92063492],
       [0.87847222],
       [0.86458333],
       [0.88095238],
       [0.87996032],
       [0.94444444],
       [0.99355159],
       [0.99553571],
       [0.96378968],
       [0.93650794],
       [0.99355159],
       [0.89831349],
       [0.9360119 ],
       [0.79662698],
       [0.88690476],
       [0.78819444],
       [0.81597222],
       [0.78025794],
       [0.69246032],
       [0.59474206],
       [0.61805556],
       [0.51984127],
       [0.48263889],
       [0.63194444],
       [0.53422619],
       [0.56001984],
       [0.4702381 ],
       [0.51339286],
       [0.4156746 ],
       [0.32936508],
       [0.33333333],
       [0.29365079],
       [0.2718254 ],
       [0.24652778],
       [0.27480159],
       [0.21130952],
       [0.25545635],
       [0.26140873],
       [0.31894841],
       [0.30208333],
       [0.371

In [139]:
x_data

array([[[0.74801587, 0.23489011],
        [0.77926587, 0.24313187],
        [0.76785714, 0.25      ],
        [0.79613095, 0.25480769]],

       [[0.77926587, 0.24313187],
        [0.76785714, 0.25      ],
        [0.79613095, 0.25480769],
        [0.83333333, 0.26167582]],

       [[0.76785714, 0.25      ],
        [0.79613095, 0.25480769],
        [0.83333333, 0.26167582],
        [0.77529762, 0.26098901]],

       ...,

       [[0.08333333, 0.01510989],
        [0.07440476, 0.01236264],
        [0.11111111, 0.00686813],
        [0.10069444, 0.00480769]],

       [[0.07440476, 0.01236264],
        [0.11111111, 0.00686813],
        [0.10069444, 0.00480769],
        [0.11706349, 0.00206044]],

       [[0.11111111, 0.00686813],
        [0.10069444, 0.00480769],
        [0.11706349, 0.00206044],
        [0.10714286, 0.        ]]])

In [117]:
#Split data
#x_data
train_size = int(len(x_data)*0.8)
test_size_x = len(x_data) - train_size
train_data_x = x_data[:train_size]
test_data_x =  x_data[train_size:]

print(train_data_x.shape)
print(test_data_x.shape)

#y_data
train_size = int(len(y_data)*0.8)
test_size_y = len(y_data) - train_size
train_data_y = y_data[:train_size]
test_data_y =  y_data[train_size:]
print(train_data_y.shape)
print(test_data_y.shape)

(390, 4, 2)
(98, 4, 2)
(390, 1)
(98, 1)


In [118]:
train_data_x.shape[1]
train_data_x.shape[2]

2

In [174]:
model = Sequential()

model.add(LSTM(units = 256, return_sequences = True, input_shape=(train_data_x.shape[1], train_data_x.shape[2])))
model.add(Dropout(0.2))

#model.add(LSTM(units = 256, return_sequences = True))
#model.add(Dropout(0.2))

model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 256))
model.add(Dropout(0.2))

model.add(Dense(units = 1))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')  

In [175]:
#model fit
history = model.fit(train_data_x, train_data_y, epochs = 200, batch_size=80)

Epoch 1/200
5/5 [==============================] - 8s 73ms/step - loss: 0.2086
Epoch 2/200
5/5 [==============================] - 0s 74ms/step - loss: 0.0402
Epoch 3/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0249
Epoch 4/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0147
Epoch 5/200
5/5 [==============================] - 0s 76ms/step - loss: 0.0127
Epoch 6/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0125
Epoch 7/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0081
Epoch 8/200
5/5 [==============================] - 0s 73ms/step - loss: 0.0081
Epoch 9/200
5/5 [==============================] - 0s 70ms/step - loss: 0.0074
Epoch 10/200
5/5 [==============================] - 0s 71ms/step - loss: 0.0074
Epoch 11/200
5/5 [==============================] - 0s 75ms/step - loss: 0.0067
Epoch 12/200
5/5 [==============================] - 0s 87ms/step - loss: 0.0066
Epoch 13/200
5/5 [==============================]

In [176]:
#Get predictions
y_predicted = model.predict(test_data_x)
y_predicted.shape

4/4 [==============================] - 1s 12ms/step


(98, 1)

In [177]:
data = scaled_dataset[:98,:]
y_pred_act = np.concatenate((y_predicted,data[:98,1:]),axis=1)
y_test_act = np.concatenate((test_data_y,data[:98,1:]),axis=1)


In [178]:
Y_tested = scaler.inverse_transform(y_test_act)
Y_predicted = scaler.inverse_transform(y_pred_act)

Y_predicted = Y_predicted[:,0:1]
Y_tested = Y_tested[:,0:1]
Y_tested.shape

(98, 1)

In [179]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_tested,Y_predicted)

rmse = np.sqrt(mse)
rmse

99.54956623215662